# Active Airport

In [1]:
# %load imports.py
# %%writefile imports.py

# https://towardsdatascience.com/how-to-effortlessly-optimize-jupyter-notebooks-e864162a06ee
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from functions import replace_df_ax_name, find_missing_values, path_checker, df_location_data, missing_location



# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns

# import chart_studio.plotly as py
# import cufflinks as cf

# from plotly.offline import plot, iplot, init_notebook_mode, download_plotlyjs

# init_notebook_mode(connected=True)
# cf.go_offline()
# pd.set_option("display.max_rows", 20)
# pd.set_option("display.max_columns", 20)

# small_fint_size = 14

In [2]:
def air_melter(df):
    df_melt = df.melt(
        id_vars=["airport", "type of traffic", "location", "point", "latitude", "longitude", "altitude"],
        var_name="date",
        value_name="passengers").sort_values(
        ["airport", "type of traffic", "location", "point", "latitude", "longitude", "altitude", "passengers"]).reset_index(drop=True)
    
    if "date" in df_melt:
        df_melt["date"] = pd.to_datetime(df_melt["date"])
    
    return df_melt

## Import data

In [3]:
df = pd.read_csv("passenger_data.csv", delimiter=";", header=1).drop(["domestic/international flights", "passenger group"], axis=1)
df = df.sort_values(by="airport")
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,airport,type of traffic,Passengers 2010M10,Passengers 2010M11,Passengers 2010M12,Passengers 2011M01,Passengers 2011M02,Passengers 2011M03,Passengers 2011M04,Passengers 2011M05,...,Passengers 2019M12,Passengers 2020M01,Passengers 2020M02,Passengers 2020M03,Passengers 2020M04,Passengers 2020M05,Passengers 2020M06,Passengers 2020M07,Passengers 2020M08,Passengers 2020M09
0,Alta,Non-scheduled passenger flights,265,2,0,0,0,142,0,0,...,0,0,0,0,4,134,330,328,336,54
1,Alta,All commercial flights,30314,25873,22914,23369,23484,29224,28631,32310,...,24943,27644,27778,16487,4221,6229,13248,24120,20621,19571
2,Alta,Freight,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alta,"Helicopter, continental shelf",0,0,0,0,0,0,10,0,...,0,0,0,0,0,16,49,27,0,0
4,Alta,"Helicopter, other",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preprocessing

In [5]:
df = replace_df_ax_name(df, "Passengers ", "", 1)
df = replace_df_ax_name(df, "M", "-", 1)
df = replace_df_ax_name(df, "-", "d_to_datetime", 1)

In [6]:
df.head()

,airport,type of traffic,2010-10-01 00:00:00,2010-11-01 00:00:00,2010-12-01 00:00:00,2011-01-01 00:00:00,2011-02-01 00:00:00,2011-03-01 00:00:00,2011-04-01 00:00:00,2011-05-01 00:00:00,...,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00
0,Alta,Non-scheduled passenger flights,265,2,0,0,0,142,0,0,...,0,0,0,0,4,134,330,328,336,54
1,Alta,All commercial flights,30314,25873,22914,23369,23484,29224,28631,32310,...,24943,27644,27778,16487,4221,6229,13248,24120,20621,19571
2,Alta,Freight,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alta,"Helicopter, continental shelf",0,0,0,0,0,0,10,0,...,0,0,0,0,0,16,49,27,0,0
4,Alta,"Helicopter, other",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
find_missing_values(df)

Instances of missing data: 0
Columns with missing data: 0
Column names with missing data: None


### Wide data

In [8]:
path = "df_geo.csv"
read = True

if read:
    if path_checker(path):
        df = pd.read_csv(path)
        if "date" in df:
            df["date"] = pd.to_datetime(df["date"])
else:
    if df is None:
        print("You need a DataFrame to export.")
    else:
        df_location_data(df=df, search_col="airport")
        df.to_csv(f'{path}', index=False)

'df_geo.csv' is a file


In [9]:
df.head()

,airport,type of traffic,2010-10-01 00:00:00,2010-11-01 00:00:00,2010-12-01 00:00:00,2011-01-01 00:00:00,2011-02-01 00:00:00,2011-03-01 00:00:00,2011-04-01 00:00:00,2011-05-01 00:00:00,...,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,location,point,latitude,longitude,altitude
0,Alta,Non-scheduled passenger flights,265,2,0,0,0,142,0,0,...,134,330,328,336,54,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.08254,0.0
1,Alta,All commercial flights,30314,25873,22914,23369,23484,29224,28631,32310,...,6229,13248,24120,20621,19571,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.08254,0.0
2,Alta,Freight,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.08254,0.0
3,Alta,"Helicopter, continental shelf",0,0,0,0,0,0,10,0,...,16,49,27,0,0,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.08254,0.0
4,Alta,"Helicopter, other",0,0,0,0,0,0,0,0,...,0,0,0,0,0,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.08254,0.0


In [10]:
find_missing_values(df)

Instances of missing data: 70
Columns with missing data: 5
Column names with missing data: location, point, latitude, longitude, and altitude


In [11]:
miss_col, miss_row, miss_only = missing_location(df)

In [12]:
miss_only

,location,point,latitude,longitude,altitude
147,NaN,NaN,NaN,NaN,NaN
148,NaN,NaN,NaN,NaN,NaN
149,NaN,NaN,NaN,NaN,NaN
150,NaN,NaN,NaN,NaN,NaN
151,NaN,NaN,NaN,NaN,NaN
152,NaN,NaN,NaN,NaN,NaN
153,NaN,NaN,NaN,NaN,NaN
245,NaN,NaN,NaN,NaN,NaN
246,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN


In [13]:
miss_col

,location,point,latitude,longitude,altitude
0,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.082540,0.0
1,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.082540,0.0
2,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.082540,0.0
3,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.082540,0.0
4,"Alta, Troms og Finnmark, Norge","(70.04962755, 23.08254009804839, 0.0)",70.049628,23.082540,0.0
...,...,...,...,...,...
359,"Ørsta/Volda lufthamn, Hovden, Torvmyrane, Hovd...","(62.17820605, 6.068381079971115, 0.0)",62.178206,6.068381,0.0
360,"Ørsta/Volda lufthamn, Hovden, Torvmyrane, Hovd...","(62.17820605, 6.068381079971115, 0.0)",62.178206,6.068381,0.0
361,"Ørsta/Volda lufthamn, Hovden, Torvmyrane, Hovd...","(62.17820605, 6.068381079971115, 0.0)",62.178206,6.068381,0.0
362,"Ørsta/Volda lufthamn, Hovden, Torvmyrane, Hovd...","(62.17820605, 6.068381079971115, 0.0)",62.178206,6.068381,0.0


In [14]:
miss_row

,airport,type of traffic,2010-10-01 00:00:00,2010-11-01 00:00:00,2010-12-01 00:00:00,2011-01-01 00:00:00,2011-02-01 00:00:00,2011-03-01 00:00:00,2011-04-01 00:00:00,2011-05-01 00:00:00,...,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,location,point,latitude,longitude,altitude
147,Mo i Rana Røssvold,"Helicopter, continental shelf",0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
148,Mo i Rana Røssvold,Other commercial flights,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
149,Mo i Rana Røssvold,Non-scheduled passenger flights,0,0,0,8,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
150,Mo i Rana Røssvold,Scheduled passenger flights,12566,12055,9297,9673,10853,11666,9710,12730,...,0,4240,9484,7532,8357,NaN,NaN,NaN,NaN,NaN
151,Mo i Rana Røssvold,All commercial flights,12566,12055,9297,9681,10853,11666,9710,12730,...,0,4240,9484,7532,8357,NaN,NaN,NaN,NaN,NaN
152,Mo i Rana Røssvold,"Helicopter, other",0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
153,Mo i Rana Røssvold,Freight,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
245,Skien Geitryggen,Non-scheduled passenger flights,0,0,3,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
246,Skien Geitryggen,Other commercial flights,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
247,Skien Geitryggen,"Helicopter, other",0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [15]:
miss_row_airport = miss_row
miss_row_airport.drop_duplicates("airport")

,airport,type of traffic,2010-10-01 00:00:00,2010-11-01 00:00:00,2010-12-01 00:00:00,2011-01-01 00:00:00,2011-02-01 00:00:00,2011-03-01 00:00:00,2011-04-01 00:00:00,2011-05-01 00:00:00,...,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,location,point,latitude,longitude,altitude
147,Mo i Rana Røssvold,"Helicopter, continental shelf",0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
245,Skien Geitryggen,Non-scheduled passenger flights,0,0,3,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [16]:
path = "df_geo_manual.csv"
read = True

if read:
    if path_checker(path):
        df = pd.read_csv(path)
        if "date" in df:
            df["date"] = pd.to_datetime(df["date"])
else:
    if df is None:
        print("You need a DataFrame to export.")
    else:

        from geopy.geocoders import Nominatim
        from geopy.point import Point

        geolocator = Nominatim(user_agent="my_geocoder")
        location = geolocator.reverse

        df.loc[df["airport"] == "Mo i Rana Røssvold", "latitude"] = 66.3646621704102
        df.loc[df["airport"] == "Mo i Rana Røssvold", "longitude"] = 14.3028783798218
        df.loc[df["airport"] == "Mo i Rana Røssvold", "altitude"] = 0.0

        df.loc[df["airport"] == "Skien Geitryggen", "latitude"] = 59.18429939776701
        df.loc[df["airport"] == "Skien Geitryggen", "longitude"] = 9.569653883827625
        df.loc[df["airport"] == "Skien Geitryggen", "altitude"] = 0.0

        mask = df[(df["airport"] == ("Mo i Rana Røssvold"))
                  | (df["airport"] == ("Skien Geitryggen"))]

        df.loc[mask.index, "point"] = [
            ', '.join(str(x) for x in y)
            for y in map(tuple, df.loc[mask.index, ["latitude", "longitude"]].values)
        ]

        df.loc[mask.index, "location"] = df.loc[mask.index, "point"].apply(location)

        df["location"] = df["location"].apply(str)

        # df = df.drop(['location', "altitude"], axis=1)
        
        df.to_csv(f'{path}', index=False)

'df_geo_manual.csv' is a file


In [17]:
# df[(df["airport"] == ("Mo i Rana Røssvold")) | (df["airport"] == ("Skien Geitryggen"))]

In [18]:
find_missing_values(df)

Instances of missing data: 0
Columns with missing data: 0
Column names with missing data: None


### Long data

In [20]:
path = "df_melt.csv"
read = False

if read:
    if path_checker(path):
        df_melt = pd.read_csv(path)
        if "date" in df_melt:
            df_melt["date"] = pd.to_datetime(df_melt["date"])
else:
    if df is None:
        print("You need a DataFrame to export.")
    else:
        df_melt = df.melt(id_vars=[
            "airport", "type of traffic", "location", "point", "latitude", "longitude",
            "altitude"
        ],
            var_name="date",
            value_name="passengers").sort_values([
                "airport", "type of traffic", "location", "point",
                "latitude", "longitude", "altitude", "passengers"
            ]).reset_index(drop=True)
        if "date" in df_melt:
            df_melt["date"] = pd.to_datetime(df_melt["date"])
        
        df_melt.drop(["altitude", "point"], axis=1, inplace=True)
        
        #df_melt['latitude'] = df_melt['latitude'].map('{:,.2f}'.format)
        #df_melt['longitude'] = df_melt['longitude'].map('{:,.2f}'.format)
        
        df_melt.to_csv(f'{path}', index=False)

In [ ]:
df_melt['date'] = df_melt['date'].apply(lambda x: str(x)[:-9])

In [ ]:
df_melt

In [ ]:

#df_melt['date'] = pd.to_datetime(df_melt["date"].dt.strftime('%Y-%m'))
#df_melt['date'] = df_melt['date'].dt.strftime('%m/%d/%Y')
df_melt.sort_values(by=["airport", "date"])
df_melt['date']=df_melt['date'].astype(str).str.strip('T00:00:00')

#df_melt

In [ ]:
df_melt["date"][0:1][0]

In [ ]:
print(df_melt)

## Information choice

In [ ]:
airports = df_melt["airport"].unique().tolist()

In [ ]:
type_of_traffic = df_melt["type of traffic"].unique().tolist()

In [ ]:
years = df_melt["date"].dt.year.unique().tolist()
years = sorted(years)

In [ ]:
months = df_melt["date"].dt.month.unique().tolist()
months = sorted(months)
months_alpha = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Des"]

## Testing

In [ ]:
df_melt[df_melt["airport"] == selected_airport]

In [ ]:
selected_airport = ["Oslo Gardermoen", "Kristiansand Kjevik"]

dfff = df_melt.copy()
dfff = dfff[(dfff["airport"] == selected_airport[0])]# | (dfff["airport"] == selected_airport[1])]

dfff

In [ ]:
dfff = dfff.groupby(
    ['date'])['passengers'].agg('sum').to_frame().reset_index()

dfff = dfff.sort_values(by="date")
dfff

In [ ]:
fig = px.line(dfff, x='date', y='passengers', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Raw Data",
    mode="lines", x=dfff["date"], y=dfff["passengers"],
))

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
df_airport_1[(df_airport_1["airport"] == airport_1)]
df_airport_2[(df_airport_2["airport"] == airport_2)]

In [ ]:
airport_1 = "Oslo Gardermoen"
airport_2 = "Kristiansand Kjevik"

df_airport_1 = df_melt.copy()
df_airport_2 = df_melt.copy()

df_airport_1 = df_airport_1[df_airport_1["airport"] == airport_1]
df_airport_2 = df_airport_2[df_airport_2["airport"] == airport_2]

df_airport_1 = df_airport_1.groupby(
    ['date'])['passengers'].agg('sum').to_frame().reset_index()
df_airport_1 = df_airport_1.sort_values(by="date")

df_airport_2 = df_airport_2.groupby(
    ['date'])['passengers'].agg('sum').to_frame().reset_index()
df_airport_2 = df_airport_2.sort_values(by="date")



fig_line = go.Figure()

fig_line.add_trace(go.Scatter(
    name=airport_1,
    mode="lines", x=df_airport_1["date"], y=df_airport_1["passengers"],
))

fig_line.add_trace(go.Scatter(
    name=airport_2,
    mode="lines", x=df_airport_2["date"], y=df_airport_2["passengers"],
))

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
airport_1 = "Oslo Gardermoen"
airport_2 = "Bergen Flesland"
selected_traffic = "All commercial flights"

df_airport_1 = df_melt[df_melt["type of traffic"] == selected_traffic].copy()

df_airport_2 = df_melt[df_melt["type of traffic"] == selected_traffic].copy()

In [ ]:
df_airport_1 = df_airport_1[df_airport_1["airport"] == airport_1]
df_airport_2 = df_airport_2[df_airport_2["airport"] == airport_2];

In [ ]:
df_airport_1

In [ ]:
df_airport_1 = df_airport_1.groupby(
    ['date'])['passengers'].agg('sum').to_frame().reset_index()
df_airport_1 = df_airport_1.sort_values(by="date")
df_airport_2 = df_airport_2.groupby(
    ['date'])['passengers'].agg('sum').to_frame().reset_index()
df_airport_2 = df_airport_2.sort_values(by="date")

In [ ]:
fig_line = go.Figure()

fig_line.add_trace(go.Scatter(
    name=airport_1,
    mode="lines", x=df_airport_1["date"], y=df_airport_1["passengers"],
))

fig_line.add_trace(go.Scatter(
    name=airport_2,
    mode="lines", x=df_airport_2["date"], y=df_airport_2["passengers"],
))

fig_line.update_xaxes(rangeslider_visible=True)
fig_line.show()

In [ ]:
selected_traffic = "All commercial flights"
selected_airport = ["Oslo Gardermoen", "Bergen Flesland", "Kristiansand Kjevik"]

df_airport = df_melt.copy()

df_airport = df_melt[df_melt["type of traffic"] == selected_traffic]

df_airport = df_airport[df_airport['airport'].isin(selected_airport)]

airport_range = df_airport["airport"].unique().tolist()

n_airports: list = []
for a in airport_range:
    n_airports.append(df_airport[df_airport["airport"] == a])
    
agg_airports: dict = {}
for a in range(len(airport_range)):
    n_airports[a] = n_airports[a].groupby(['date'])['passengers'].agg('sum').to_frame().reset_index()
    agg_airports[airport_range[a]] = n_airports[a].sort_values(by="date")
    
fig_line = go.Figure()

for k, v in agg_airports.items():
    fig_line.add_trace(go.Scatter(
        name=k,
        mode="lines", x=v["date"], y=v["passengers"],
    ))

fig_line.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="Month", step="month", stepmode="backward"),
            dict(count=6, label="6 Months", step="month", stepmode="backward"),
            dict(count=1, label="Today", step="year", stepmode="todate"),
            dict(count=1, label="Year", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
)

fig_line.show()